In [1]:
from pprint import pprint

In [ ]:
# class User
    # def like


In [3]:
# Init - Choose Categories

# Carl chose 2 categories
# Always adds up to 100
CarlInterests = {
    'DevOps': 45,
    'Machine Learning': 45,
    'Random': 10,
}

print(CarlInterests)

{'DevOps': 45, 'Machine Learning': 45, 'Random': 10}


In [ ]:
# Updating Weights

# Is called when watching a video

Like = 0.1
Share = 0.2
Watch = 0.1
Loop = 0.05

def update_weight(user_dict_interest, video_tag):



In [ ]:
# Auto-Adding Mechanism